In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
from abc import ABC, abstractmethod
plt.rcParams["font.family"] = "Arial"

In [2]:
np.random.seed(42)
random.seed(42)

## Tool

A hardware rental store has a catalog of 20 different tools to rent, spread across 5 different categories (Painting, Concrete, Plumbing, Woodwork, Yardwork). Each tool has a unique name (e.g. “Paint Tool 1”) and belongs to a specific category; the price per day to rent a tool varies by category. You may decide on the pricing of the rental categories.

In [3]:
class Tool:
    def __init__(self, name, price, category, available):
        self.name = name
        self.price = price
        self.category = category
        self.available = available 
        
    def __repr__(self):
        return (f"{self.__class__.__name__}"
                f"({self.name}, ${self.price}, "
                f"{self.category}, {self.available})")
    

## ToolCollection as inventory

In [12]:
np.random.seed(42)
random.seed(42)

class ToolCollection:
    def __init__(self, num_tools=20):
        self.num_tools = num_tools
        self.tool_categories = ["Painting", "Concrete", "Plumbing", "Woodwork", "Yardwork"]
        self.tool_prices = [1, 2, 3, 4, 5]
        self.num_tool_categories = len(self.tool_categories)
        
        self.available_tools = []
        self.rented_tools = []
        
        self.__generate_tools()
        
    def __generate_tools(self):
        for tool_i in range(self.num_tools):
            random_idx = np.random.randint(0, self.num_tool_categories)
            name = f"T{tool_i+1:02d}-{self.tool_categories[random_idx]}"
            price = self.tool_prices[random_idx]
            category = self.tool_categories[random_idx]
            available = True
            tool_obj = Tool(name, price, category, available)
            self.available_tools.append(tool_obj)
    
    def __len__(self):
        return len(self.available_tools)
        
    def rent(self, chosen_tools): # MAYBE FOR STORE
        for chosen_tool in chosen_tools:
            for tool_i, tool in enumerate(self.available_tools):
                if tool.name == chosen_tool.name:
                    tool.available = False
                    self.rented_tools.append(chosen_tool)
                    del self.available_tools[tool_i]
                    break
                
    def __call__(self):
        return self.available_tools
    
    def restock(self, tools):
        for tool in tools:
            tool.available = True
            self.available_tools.append(tool)
    
    def print_tool_list(self):
        for t in self.available_tools:
            print(t)
            
    def __contains__(self, other):
        pass
    
    def __iter__(self):
        for tool in self.available_tools:
            yield tool

## Customers

This store has 10 customers; each customer has a unique name and is associated with one of three types. Casual customers rent one or two tools for one or two nights. Business customers always rent three tools for seven nights. Regular customers will rent one to three tools each time they visit for 3 to 5 nights.

In [13]:
class Customer:
    def __init__(self, name, customer_type, num_tools_wanted, num_nights, num_tools_rented):
        self.name = name
        self.customer_type = customer_type
        self.num_tools_wanted = num_tools_wanted
        self.num_nights = num_nights
        self.num_tools_rented = num_tools_rented
        
    def __repr__(self):
        return (f"{self.__class__.__name__}"
                f"({self.name}, {self.customer_type}, "
                f"{self.num_tools_wanted}, {self.num_nights}), NUM_TOOLS_RENTED={self.num_tools_rented}")

## CustomerCollection (generator)

In [20]:
np.random.seed(42)
random.seed(42)

class CustomerCollection:
    
    def __init__(self, num_customers=10, num_customer_types=3):
        self.num_customers = num_customers
        self.num_customer_types = num_customer_types
        
        self.customer_types = ["Casual", "Business", "Regular"]
        self.num_tools = [[1,2], [3], [1,2,3]]
        self.num_nights = [[1,2], [7], [3,4,5]]

        self.customer_objects = []
        
        self.__generate_customers()
        
    def __generate_customers(self):
        for customer_i in range(self.num_customers):
            random_idx = np.random.randint(0, self.num_customer_types)
 
            customer_params = {
                "name"             : f"C{customer_i+1:02d}-{self.customer_types[random_idx][0]}", 
                "customer_type"    : self.customer_types[random_idx], 
                "num_tools_wanted" : random.sample(self.num_tools[random_idx], 1)[0], 
                "num_nights"       : random.sample(self.num_nights[random_idx], 1)[0], 
                "num_tools_rented" : 0
            }

            customer_obj = Customer(**customer_params)
            self.customer_objects.append(customer_obj)
            
    def return_tool(self, customer_name, num_tools_returned):
        for customer in self.customer_objects:
            if customer.name == customer_name:
                customer.num_tools_rented -= num_tools_returned
                break
                
    def get_customers(self):
        customer_pool = [customer for customer in self.customer_objects if customer.num_tools_rented < 3]  # Only customers with 0, 1, 2 active tools
        todays_customers = random.sample(customer_pool, np.random.randint(len(customer_pool)))        
        return todays_customers
                
    def print_customer_list(self):
        for c in self.customer_objects:
            print(c)
            
    def __contains__(self, other):
        pass
    
    def __iter__(self):
        for customer in self.customer_objects:
            yield customer

## Rental

Each time a customer comes into the store, a Rental is created that will keep track of what tools they rented and how many nights they will keep the tools.

A customer can have more than one active rental. That is, they can show up on day 1
and rent 1 tool for 5 nights. They can then show up on day 2 and rent another tool for 4
nights. As long as they do not have more than 3 tools rented, they are allowed to have
multiple rentals.

In [21]:
class Rental:
    def __init__(self, customer_name, tools_rented, 
                 num_rent_nights, total_price,
                 day_rented, day_due, returned):
        
        self.customer_name = customer_name
        self.tools_rented = tools_rented
        self.num_rent_nights = num_rent_nights
        self.total_price = total_price
        self.day_rented = day_rented
        self.day_due = day_due
        self.returned = returned
        
    def set_returned(self):
        self.returned = True
        
    def __repr__(self):
        return (f"{self.__class__.__name__}, Customer: {self.customer_name}, "
                f"({self.tools_rented}, RentDate: {self.day_rented+1}, "
                f"DueDate: {self.day_due+1}, TotalPrice: ${self.total_price}, RETURNED: {self.returned})")

## RentalCollection

Keep a history of all completed and active rentals.

In [22]:
class RentalCollection:
    def __init__(self):
        self.completed_rentals = []
        self.active_rentals = []
        
    def create_rental(self, customer, chosen_tools, total_price, day_i):
        # For a given customer, make rental record
        rental_params = {
                "customer_name"   : customer.name, 
                "tools_rented"    : chosen_tools, 
                "num_rent_nights" : customer.num_nights, 
                "total_price"     : total_price,
                "day_rented"      : day_i, 
                "day_due"         : day_i + customer.num_nights, 
                "returned"        : False
            }
            
        rental = Rental(**rental_params)
        return rental

    def update_rentals(self, rental):
        # Given a rental object, check its returned status and put in appropriate list
        if rental.returned:
            self.completed_rentals.append(rental)
        else:
            self.active_rentals.append(rental)
    
    def __repr__(self):
        return (f"Completed rentals: {self.completed_rentals} \n"
                f"Active rentals: {self.active_rentals} \n")

## Transaction

HAVE ACCESS TO TOOLS, RENTALS, AND CUSTOMERS? for a given transaction, they interact. 

CAN BE A FACADE FOR THESE CLASSES - ENFORCE SOME METHODS TO BE DONE BY THE CLASSES
FOR EX: CUSTOMERS SHOULD HAVE RETURN()

STORE == CLIENT? OR MAIN?

Client just needs to see the transaction - some output from transaction: existing rentals and current inventory

## Store

The store keeps track of the existing rentals along with the current inventory of the store. As such, when it has zero rentals, there will be 20 tools in its inventory. When it has zero tools in its inventory, it will have multiple rentals that between them account for all 20 tools.

In [61]:
class Store:
    def __init__(self, tools, rental_collection, customers, total_money, num_avail):
        self.tools = tools  # Inventory
        self.rental_collection = rental_collection # Rentals
        self.total_money = total_money
        self.num_avail = num_avail
    
    def get_num_tools_available(self):
        self.num_avail = len(self.tools)
        return self.num_avail
    
    def get_available_tools(self):
        return tools()
    
    def update_money(self, today_money): # TRANSACION: RETURN TODAY'S MONEY?
        self.total_money += today_money
        
    def process_returns(self, day_i):  # TRANSACTION? 
        for rental_receipt in rental_collection.active_rentals:
            if rental_receipt.day_due == day_i:
                rental_receipt.set_returned()
                customers.return_tool(rental_receipt.customer_name, len(rental_receipt.tools_rented))
                tools.restock(rental_receipt.tools_rented)
                print(f"RETURN: {rental_receipt.customer_name}")
                
    def update_customers(self, customer): # TRANSACTION? 
        customer.num_tools_rented += customer.num_tools_wanted
                
    def rent_tools(self, todays_customers, day_i): # TRANSACTION? 
        for customer_i, customer in enumerate(todays_customers):
            if customer.num_tools_wanted <= self.num_avail and customer.num_tools_wanted + customer.num_tools_rented <= 3:

                available_tools = tools()
                chosen_tools = random.sample(available_tools, customer.num_tools_wanted)
                tools.rent(chosen_tools)
                total_price = sum([tool.price for tool in chosen_tools] * customer.num_nights)

                # Make a Rental record for this customer & update rental records
                rental = rental_collection.create_rental(customer, chosen_tools, total_price, day_i)
                rental_collection.update_rentals(rental)

                # Update total earning
                self.update_money(rental.total_price)

                # Each day, update num_tools_rented for Customer objects that rent
                self.update_customers(customer)

                # Update number of tools in inventory
                self.num_avail = self.get_num_tools_available()

                print(f"{customer_i}--- {customer}")
                print(rental)
                
    def output_data(self):
        pass


## Report 

At the end of the simulation, the program will produce a report that includes the following information:
- the number of tools currently in the store along with a list of their names
- the amount of money the store made during the 35 days (including any rentals
that occurred on the 35th day)
- a list of all the completed rentals including which tools were rented by which
customer for how many days along with the total amount of that rental
- a list of all the active rentals that includes all of the information listed in the
previous bullet

In [62]:
# class Report:
#     def write_report(self, num_tools, tools, total_money, 
#                      completed_rentals, active_rentals):
#         print(f"Number of tools currently in store: {num_tools} \n"
#               f"Tools currently in store: {tools} \n"
#               f"Completed rentals: {completed_rentals} \n"
#               f"Active rentals: {active rentals} \n")
        

## Main

In [63]:
np.random.seed(42)
random.seed(42)

num_days = 35

# Instantiate ToolCollection object
tools = ToolCollection()

# Instantiate CustomerCollection object 
customers  = CustomerCollection()

# Instantiate RentalCollection object
rental_collection = RentalCollection()

# Instantiate Store object
store = Store(tools, rental_collection, customers, total_money=0, num_avail=20)

for day_i in range(0, num_days):
    
    print(f"---------------- DAY {day_i+1} ----------------")
    
    #------- Rental returns
    store.process_returns(day_i)

    #------- Open store, check inventory   
    num_tools_available_in_store = store.get_num_tools_available()
    print(f"\nNumber of tools available = {num_tools_available_in_store}\n")

    if num_tools_available_in_store == 0: 
        print("NO TOOLS LEFT TODAY")
        continue

    #------- Customers come in if store isn't empty
    todays_customers = customers.get_customers()
    store.rent_tools(todays_customers, day_i)   

    print(f"\nMoney earned to date: ${store.total_money}\n")
    

---------------- DAY 1 ----------------

Number of tools available = 20

0--- Customer(C09-B, Business, 3, 7), NUM_TOOLS_RENTED=3
Rental, Customer: C09-B, ([Tool(T08-Plumbing, $3, Plumbing, False), Tool(T15-Woodwork, $4, Woodwork, False), Tool(T09-Plumbing, $3, Plumbing, False)], RentDate: 1, DueDate: 8, TotalPrice: $70, RETURNED: False)
1--- Customer(C07-R, Regular, 1, 3), NUM_TOOLS_RENTED=1
Rental, Customer: C07-R, ([Tool(T01-Woodwork, $4, Woodwork, False)], RentDate: 1, DueDate: 4, TotalPrice: $12, RETURNED: False)

Money earned to date: $82

---------------- DAY 2 ----------------

Number of tools available = 16

0--- Customer(C03-C, Casual, 1, 1), NUM_TOOLS_RENTED=1
Rental, Customer: C03-C, ([Tool(T14-Concrete, $2, Concrete, False)], RentDate: 2, DueDate: 3, TotalPrice: $2, RETURNED: False)
1--- Customer(C07-R, Regular, 1, 3), NUM_TOOLS_RENTED=2
Rental, Customer: C07-R, ([Tool(T03-Plumbing, $3, Plumbing, False)], RentDate: 2, DueDate: 5, TotalPrice: $9, RETURNED: False)
2--- Custo

In [ ]:
class A:
    def hello(self):
        print("I'm A")
        print(super())
        
    def test(self):
        print("I'm A")
        print(super())

class B(A):
    def hello(self):
        print("I'm B")
        print(super())
        print(A.test())
        
    def test(self):
        print("I'm B test")
        print(super())

class C(A):
    def hello(self):
        print("I'm C")
        print(super())
        global xxx
        xxx = super
        print(super().test())
        
    def test(self):
        print("I'm C")
        print(super())

class D(C, B):
    def hello(self):
        print("I'm D")
        print(super())
        super().hello()
        
    def test(self):
        print("I'm D")
        print(super())

In [ ]:
d = D()

In [ ]:
d.hello()

In [ ]:
help(D)